# REDE DE CITAÇÕES - SCOPUS - LOGIC

In [1]:
# CARREGANDO BIBLIOTECAS
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import OpenSSL.SSL
import getpass

#!pip install scopus requests
#!pip install update scopus
import scopus as sc

from selenium import webdriver
from bs4 import BeautifulSoup

from pymongo import MongoClient
import pandas as pd
import pickle

from math import *

import logging
import os
import shutil
import send2trash
import time
import itertools
import gc 

import configparser as cf
import importlib

path_atual = os.getcwd()
print(path_atual)
path_download = '../../Downloads/'

C:\Users\Walter\Documents\WOS_CitationNetworks


## 1. No WoS, escolher uma área de pubicação e obter a listagem de periódicos.

Usando o Web of Science, obter uma listagem de todos os periódicos publicados na categoria escolhida (**MATHEMATICS**). A fim de filtrar os resultados, foi escolhida uma subárea: **LOGIC**.

* Em `Advanced Search` do WoS, pesquisar: `WC=(MATHEMATICS)` e selecionar a opção de apenas artigos. São encontrados 1.268.522 artigos. 


* Em seguida, selecionar o filtro de categoria do WoS, `LOGIC`. Agora, são eoncontrados 13.030 artigos. 



* No campo de filtros, tem a opção `Source Titles`. Ir em `more options / values...`. Com a página atualizada, ir em ` Analyze results`.


* Novamente, escolher a opção `Source Titles`. 


* No final da página, selecionar a opção `All data rows (up to 100,000)` e realizar o download. 

![](figuras/journalsVis_LOGIC.jpg)

In [2]:
file_journals = pd.read_csv('journals/SourceTitles_LOGIC.txt', sep='\t',index_col=False)
file_journals.index += 1 
file_journals = file_journals[:-1]

In [3]:
file_journals.head()

,Source Titles,records,% of 13030
1,JOURNAL OF SYMBOLIC LOGIC,3944.0,30.269
2,ANNALS OF PURE AND APPLIED LOGIC,2232.0,17.130
3,MATHEMATICAL LOGIC QUARTERLY,1322.0,10.146
4,ARCHIVE FOR MATHEMATICAL LOGIC,1126.0,8.642
5,JOURNAL OF ALGORITHMS,852.0,6.539


In [4]:
file_journals.tail()

,Source Titles,records,% of 13030
15,REPORTS ON MATHEMATICAL LOGIC,65.0,0.499
16,LECTURE NOTES IN MATHEMATICS,21.0,0.161
17,STOCHASTIC CALCULUS WITH INFINITESIMALS,11.0,0.084
18,CONSTRUCTIVE COMMUTATIVE ALGEBRA PROJECTIVE MO...,5.0,0.038
19,MODEL THEORY IN ALGEBRA ANALYSIS AND ARITHMETIC,5.0,0.038


In [5]:
SourceTitles = list(file_journals['Source Titles'])

In [6]:
file_years = pd.read_csv('journals/PublicationYears_LOGIC.txt', sep='\t',index_col=False)
file_years.index += 1 
file_years = file_years[:-2]

In [7]:
file_years.head()

,Publication Years,records,% of 13030
1,2019,141.0,1.082
2,2018,453.0,3.477
3,2017,531.0,4.075
4,2016,523.0,4.014
5,2015,509.0,3.906


In [8]:
file_years.tail()

,Publication Years,records,% of 13030
50,1970,39.0,0.299
51,1969,53.0,0.407
52,1968,45.0,0.345
53,1967,37.0,0.284
54,1966,42.0,0.322


In [9]:
PublicationYears = list(file_years['Publication Years'])

## 2. Usando a API do Scopus, pesquisar os artigos publicados por cada periódico.

Usando a função [**ScopusSearch**](https://scopus.readthedocs.io/en/stable/examples/ScopusSearch.html), buscamos pelo nome de cada periódico (`SourceTitles`) em um intervalo de tempo pré-definido de 1960 a 2018. 

* O nome do periódico deve ser colocado entre aspas duplas para ser pesquisado como uma string, caso contrário será pesquisado como uma interseção das palavras que estão contidas no nome. 

` documents = sc.ScopusSearch('SRCTITLE ("' + str(journal) + '")  AND  PUBDATETXT (' + str(year) + ') AND DOCTYPE(ar)', refresh=True, apiKey=keys[k])`

`documents = sc.ScopusSearch('SRCTITLE ( "JOURNAL OF SYMBOLIC LOGIC	" )  AND  PUBDATETXT (2018) AND DOCTYPE(ar)', refresh=True)`

Como podemo gerar apenas 10 chaves de uma só vez, todas foram armazenadas em uma lista para ser usada durante o processo de extração. Caso o valor seja excedido, a extração irá parar e novas chaves derão ser geradas [manualmente](https://dev.elsevier.com/apikey/manage) e substituídas na lista `keys`. 

In [10]:
def createKeys(password,email):
    
    driver = webdriver.Chrome(executable_path = r'/Users/Walter/Documents/chromedriver')

    scopus_api_link = 'https://dev.elsevier.com/apikey/manage'
    driver.get(scopus_api_link)

    driver.find_element_by_xpath('//*[@id="bdd-email"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="bdd-elsPrimaryBtn"]').click()
    driver.find_element_by_xpath('//*[@id="bdd-password"]').send_keys(password)
    driver.find_element_by_xpath('//*[@id="bdd-elsPrimaryBtn"]').click()
    
    # deletar todas as dez chaves existentes
    for api in range(1,11):
        driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[4]/a').click()
        driver.find_element_by_xpath('//*[@id="delregister"]').click()
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="deleteModal"]/div/div/div[3]/button[1]').click()

    # criar todas as dez chaves permitidas
    for api in range(1,11):
        driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[2]/h2/span/a').click()
        driver.find_element_by_xpath('//*[@id="projectName"]').send_keys('scopus'+str(api))
        driver.find_element_by_xpath('//*[@id="websiteURL"]').send_keys('https://github.com/anacwagner/')
        driver.find_element_by_xpath('//*[@id="apicheckbox"]/div/label').click()  
        driver.find_element_by_xpath('//*[@id="tdmcheckbox"]/div/label').click()
        driver.find_element_by_xpath('//*[@id="register"]').click()   
        
    
    soup = BeautifulSoup(driver.page_source,'lxml')
    findAll = soup.findAll('a')
    
    keys = []
    for i in range(10,20):
        keys.append(findAll[i].text)
        
    driver.quit()
    
    return keys

In [11]:
password = getpass.getpass('Password:')

Password:········


In [12]:
email = 'acwgdb@gmail.com'
#email = 'ana.wagner@fgvmail.br'

In [13]:
keys = createKeys(password,email)

C:\Users\Walter\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)


In [14]:
keys

['86c33a7c5679e09ae74597268a31e3d2',
 '4df0b90b1df6f38e6cdd867413db4645',
 'c4a028e892104e8702dacf1a9ede764c',
 '582bcb73c2154f0678d15405f02f01be',
 '02ee7c31bf2017719119f7626a94be25',
 '5f20206faef1d8ce2d1ccbfaf36c8ce6',
 '97e7c92910f1147a7cdcf49a60973473',
 '275dbc6cb9f08458475cd399e5160e04',
 '463861996177038ac90b86049c7a6724',
 '3c3d577826a5b8a905f02e6d36009807']

In [15]:
documents = sc.ScopusSearch('SRCTITLE ("' + str(SourceTitles[0]) + '")  AND  PUBDATETXT (' + str(2018) + ') AND DOCTYPE(ar)', refresh=True, apiKey=keys[0])

df_documents = pd.DataFrame(pd.DataFrame(documents.results))
df_documents.index += 1 
df_documents.head(2)

,eid,doi,pii,pubmed_id,title,subtype,creator,afid,affilname,affiliation_city,...,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
1,2-s2.0-85061941438,10.1017/jsl.2017.89,S0022481217000895,None,Profiniteness in finitely generated varieties ...,ar,Nurakunov A.,60072558;60003675,Kyrgyz National Academy of Sciences;Politechni...,Bishkek;Warsaw,...,4,None,1566-1578,© The Association for Symbolic Logic 2018. Pro...,profinite algebras | standard varieties | unde...,0,0,None,undefined,None
2,2-s2.0-85061938998,10.1017/jsl.2018.35,S002248121800035X,None,Berkeley cardinals and the structure of l(v&am...,ar,Cutolo R.,60017293,Università degli Studi di Napoli Federico II,Naples,...,4,None,1457-1476,© The Association for Symbolic Logic 2018. We...,Berkeley cardinals | choiceless large cardinal...,0,0,None,undefined,None


O resultado da pesquisa sobre `documents` nos permite ter acesso às seguintes informações:

Index(['eid', 'doi', 'pii', 'pubmed_id', 'title', 'subtype', 'creator', 'afid',
       'affilname', 'affiliation_city', 'affiliation_country', 'author_count',
       'author_names', 'author_ids', 'author_afids', 'coverDate',
       'coverDisplayDate', 'publicationName', 'issn', 'source_id', 'eIssn',
       'aggregationType', 'volume', 'issueIdentifier', 'article_number',
       'pageRange', 'description', 'authkeywords', 'citedby_count',
       'openaccess', 'fund_acr', 'fund_no', 'fund_sponsor'],
      dtype='object')

A API do Scopus para o uso de **ScopusSearch** possui limites de requests por segundo e um limite semanal.

In [16]:
# SCOPUS SEARCH
WeeklyQuota_SS =  20000
RequestSec = 6

Se tiver rodando pela primeira vez, retire os comentários das duas primeiras linhas do script abaixo e comente as duas linhas seguintes que armazenam as variáveis a cada rodada.

In [24]:
source_eids = []
dic_error_years = {}

# with open('pickles/LOGIC_eids_1.pickle', 'rb') as f:
#     source_eids, dic_error_years = pickle.load(f)

i = len(source_eids) + 1
k = 0
limit = 0

for journal in SourceTitles[i-1:]:
    
    print('\n ' + str(i) + '. Início da extração dos artigos do periódico \n' + journal)
    
    if k == 9 and limit >= 10*(WeeklyQuota_SS-20):
        print('>>> Geração de Novas Chaves!')
        keys = createKeys(password,email)
        k = 0
        limit = 0
        print('>>> Troca de Chave: ' + str(k))
        changeKeys(k,keys)

    elif limit >= (WeeklyQuota_SS-20):
        with open('scopus_infos_v2.pickle', 'wb') as f:
            pickle.dump([info_articles,info_journals], f)
        k = k + 1
        limit = 0
        print('>>> Troca de Chave: ' + str(k))
        changeKeys(k,keys)
                
    
    eids = []
    error_years = []
    for year in range(1966,2019):
        
        documents = sc.ScopusSearch('SRCTITLE ("' + str(journal) + '")  AND  PUBDATETXT (' + str(year) + ') AND DOCTYPE(ar)', refresh=True, apiKey=keys[k])

        try:
            eids.append(documents.get_eids())
            print(documents.get_results_size())
            print('Ano ' + str( year) + ' conluído.')
        except KeyError:
            try:
                time.sleep(14)
                eids.append(documents.get_eids())
                print(documents.get_results_size())
                print('Ano ' + str(year) + ' conluído.')
            except KeyError: 
                error_years.append(year)
                print(documents.get_results_size())
                print('>>> ERRO NO ANO ' + str(year) + '! <<<')
            
    source_eids.append(eids)
    dic_error_years[journal] = error_years
    i = i + 1
    limit = limit + 1
    
    with open('pickles/LOGIC_eids_1.pickle', 'wb') as f:
        pickle.dump([source_eids, dic_error_years], f)
    gc.collect()


 9. Início da extração dos artigos do periódico 
REVIEW OF SYMBOLIC LOGIC
0
Ano 1966 conluído.
0
Ano 1967 conluído.
0
Ano 1968 conluído.
0
Ano 1969 conluído.
0
Ano 1970 conluído.
0
Ano 1971 conluído.
0
Ano 1972 conluído.
0
Ano 1973 conluído.
0
Ano 1974 conluído.
0
Ano 1975 conluído.
0
Ano 1976 conluído.
0
Ano 1977 conluído.
0
Ano 1978 conluído.
0
Ano 1979 conluído.
0
Ano 1980 conluído.
0
Ano 1981 conluído.
0
Ano 1982 conluído.
0
Ano 1983 conluído.
0
Ano 1984 conluído.
0
Ano 1985 conluído.
0
Ano 1986 conluído.
0
Ano 1987 conluído.
0
Ano 1988 conluído.
0
Ano 1989 conluído.
0
Ano 1990 conluído.
0
Ano 1991 conluído.
0
Ano 1992 conluído.
0
Ano 1993 conluído.
0
Ano 1994 conluído.
0
Ano 1995 conluído.
0
Ano 1996 conluído.
0
Ano 1997 conluído.
0
Ano 1998 conluído.
0
Ano 1999 conluído.
0
Ano 2000 conluído.
0
Ano 2001 conluído.
0
Ano 2002 conluído.
0
Ano 2003 conluído.
0
Ano 2004 conluído.
0
Ano 2005 conluído.
0
Ano 2006 conluído.
0
Ano 2007 conluído.
20
Ano 2008 conluído.
21
Ano 2009 conluído.

0
Ano 2006 conluído.
0
Ano 2007 conluído.
0
Ano 2008 conluído.
0
Ano 2009 conluído.
0
Ano 2010 conluído.
8
Ano 2011 conluído.
8
Ano 2012 conluído.
4
Ano 2013 conluído.
6
Ano 2014 conluído.
0
Ano 2015 conluído.
9
Ano 2016 conluído.
5
Ano 2017 conluído.
7
Ano 2018 conluído.

 16. Início da extração dos artigos do periódico 
LECTURE NOTES IN MATHEMATICS
0
Ano 1966 conluído.
0
Ano 1967 conluído.
0
Ano 1968 conluído.
0
Ano 1969 conluído.
0
Ano 1970 conluído.
0
Ano 1971 conluído.
0
Ano 1972 conluído.
0
Ano 1973 conluído.
0
Ano 1974 conluído.
0
Ano 1975 conluído.
0
Ano 1976 conluído.
0
Ano 1977 conluído.
0
Ano 1978 conluído.
0
Ano 1979 conluído.
0
Ano 1980 conluído.
0
Ano 1981 conluído.
0
Ano 1982 conluído.
1
Ano 1983 conluído.
0
Ano 1984 conluído.
0
Ano 1985 conluído.
0
Ano 1986 conluído.
0
Ano 1987 conluído.
0
Ano 1988 conluído.
0
Ano 1989 conluído.
0
Ano 1990 conluído.
0
Ano 1991 conluído.
0
Ano 1992 conluído.
0
Ano 1993 conluído.
0
Ano 1994 conluído.
0
Ano 1995 conluído.
0
Ano 1996 conluí

In [30]:
print('Total de Chaves Utilizadas: ' + str(k+1))

Total de Chaves Utilizadas: 1


A variável `source_eids` é uma lista de listas que armazena os códigos de identificação dos artigos publicados por cada periódico em cada ano. Cada ano é uma lista dentro da lista das publicações do periódico correpondente.

In [31]:
source_eids_merged = list(itertools.chain.from_iterable(source_eids))
source_eids_merged = list(itertools.chain.from_iterable(source_eids_merged))
print(len(source_eids_merged))
print(len(set(source_eids_merged)))

18700
18700


A variável `dic_error_years` é um diconário que armazena os anos em que ocorrem os erros durante o processo (provalemnete devido a extrapolação da quantidade máxima de requests por segundo). 

In [32]:
len(dic_error_years)

19

In [33]:
dic_error_years.values()

dict_values([[], [], [], [], [1980], [], [], [], [], [], [], [], [], [], [], [], [], [], []])

Vamos rodar o script anterior (com algumas adaptações) até que o dicionário `dic_error_years2`, ao final do processo, retorne vazio.

In [34]:
# FIRST
aux_years = list(dic_error_years.values())
aux_journals = list(dic_error_years.keys())
source_eids2 = []

# # SECOND
# with open('pickles/LOGIC_eids_1.pickle', 'rb') as f:
#     source_eids, dic_error_years = pickle.load(f)
        
# aux_years = list(dic_error_years2.values())
# aux_journals = list(dic_error_years2.keys())

dic_error_years2 = {}
j = 0

for years in aux_years:
    
    j = j + 1
    
    if years != []:
        
        print('\n ' + str(j) + '. Início da extração dos artigos do periódico \n' + str(aux_journals[j-1]))

        eids = []
        error_years = []
        for year in years:

            documents = sc.ScopusSearch('SRCTITLE ("' + str(aux_journals[j-1]) + '")  AND  PUBDATETXT (' + str(year) + ') AND DOCTYPE(ar)', refresh=True, apiKey=keys[k])

            try:
                eids.append(documents.get_eids())
                print(documents.get_results_size())
                print('Ano ' + str( year) + ' conluído.')
            except KeyError:
                try:
                    time.sleep(14)
                    eids.append(documents.get_eids())
                    print(documents.get_results_size())
                    print('Ano ' + str(year) + ' conluído.')
                except KeyError: 
                    error_years.append(year)
                    print(documents.get_results_size())
                    print('>>> ERRO NO ANO ' + str(year) + '! <<<')

        source_eids2.append(eids)

        dic_error_years2[aux_journals[j-1]] = error_years
        i = i + 1

        with open('pickles/LOGIC_eids_2.pickle', 'wb') as f:
            pickle.dump([source_eids2, dic_error_years2], f)


 5. Início da extração dos artigos do periódico 
JOURNAL OF ALGORITHMS
18
Ano 1980 conluído.


In [35]:
dic_error_years2.values()

dict_values([[]])

In [36]:
source_eids_merged2 = list(itertools.chain.from_iterable(source_eids2))
source_eids_merged2 = list(itertools.chain.from_iterable(source_eids_merged2))
print(len(source_eids_merged2))
print(len(set(source_eids_merged2)))

18
18


In [37]:
all_source_eids = source_eids_merged + source_eids_merged2
all_source_eids = list(set(all_source_eids))

with open('pickles/LOGIC_eids_TOTAL.pickle', 'wb') as f:
            pickle.dump(all_source_eids, f)
        
print('Total de artigos: ' + str(len(all_source_eids)))

Total de artigos: 18718


## 3. Usando a API do Scopus, extrair e armazenar as informações de cada artigo.

As informações extraídas são armazenadas em dois diconários, um contendo as informações dos artigos (**info_articles**) e outro contendo as informações dos periódicos (**info_journals**). Essas estruturas foram pensadas para serem utilizadas no **PyMongo**!

In [17]:
with open('pickles/LOGIC_eids_TOTAL.pickle', 'rb') as f:
    all_source_eids = pickle.load(f)

In [18]:
#keys = createKeys(password,email)
keys

['86c33a7c5679e09ae74597268a31e3d2',
 '4df0b90b1df6f38e6cdd867413db4645',
 'c4a028e892104e8702dacf1a9ede764c',
 '582bcb73c2154f0678d15405f02f01be',
 '02ee7c31bf2017719119f7626a94be25',
 '5f20206faef1d8ce2d1ccbfaf36c8ce6',
 '97e7c92910f1147a7cdcf49a60973473',
 '275dbc6cb9f08458475cd399e5160e04',
 '463861996177038ac90b86049c7a6724',
 '3c3d577826a5b8a905f02e6d36009807']

In [19]:
def changeKeys(k,keys):
    config = cf.ConfigParser()
    config.read('../../../Walter/.scopus/config.ini')
    config.set("Authentication", "APIKey", keys[k])
    with open('../../../Walter/.scopus/config.ini', "w+") as configfile:
        config.write(configfile)
    print('Nova Chave: ' + config.get("Authentication", "APIKey"))

In [20]:
changeKeys(0,keys)

Nova Chave: 86c33a7c5679e09ae74597268a31e3d2


In [21]:
with open('pickles/LOGIC_INFOS.pickle', 'rb') as f:
    [info_articles, info_journals] = pickle.load(f)

In [23]:
# info_journals = {}
# info_articles = {}

limit = 0
k = 0

for eid in all_source_eids[len(info_articles):]:
    
    if k == 9 and limit >= 10*(10000-20):
        with open('pickles/LOGIC_INFOS.pickle', 'wb') as f:
            pickle.dump([info_articles,info_journals], f)
        print('>>> Geração de Novas Chaves!')
        keys = createKeys(password,email)
        k = 0
        limit = 0
        print('>>> Troca de Chave: ' + str(k))
        changeKeys(k,keys)
        
    elif limit >= (10000-20):
        with open('pickles/LOGIC_INFOS.pickle', 'wb') as f:
            pickle.dump([info_articles,info_journals], f)
        k = k + 1
        limit = 0
        print('>>> Troca de Chave: ' + str(k))
        changeKeys(k,keys)
        
        
    id_article = len(info_articles) + 1
    print('Início da extração das informações do artigo ' + str(id_article))
    
    try:
        infos = sc.AbstractRetrieval(eid , view='FULL', refresh=True)
    except sc.exception.Scopus404Error:
        time.sleep(3)
        infos = sc.AbstractRetrieval(eid , view='FULL', refresh=True)
        
    limit = limit + 1
    
    if infos.issn != None:
        
        aux_issn = infos.issn.split(' ')
        m=0
        n=1
        issn = aux_issn[0]
        
        if len(aux_issn) == 1: 
            n = 0

        if info_journals.get(aux_issn[m]) == None and info_journals.get(aux_issn[n]) == None:
            
            id_journal = len(info_journals) + 1
            info_journals[issn] = {"ID_J": id_journal,
                                   "TITLE_J": infos.publicationName}

        elif info_journals.get(aux_issn[0]) != None:
            id_journal = info_journals[issn]['ID_J']

        else:
            issn = aux_issn[1]
            id_journal = info_journals[issn]['ID_J']
            
    else: 
        issn = infos.publicationName
        if info_journals.get(issn) == None:
            id_journal = len(info_journals) + 1
            info_journals[issn] = {"ID_J": id_journal,
                                   "TITLE_J": infos.publicationName}
        else:
            id_journal = info_journals[issn]['ID_J']
            
            
    refs = infos.references
    refs_eids = []
    if refs != None:
        for r in range(len(refs)):
            refs_eids.append(refs[r][1])
            
    authors = {}
    if infos.authors != None:
        for a in range(len(infos.authors)):
            authors['author_' + str(a)] ={'auid': infos.authors[a][0],
                                          'indexed_name': infos.authors[a][1],
                                          'affiliation': infos.authors[a][-1]}
            
    try: 
        cited = infos.citedby_count
    except KeyError:
        cited = 'NaN'
    
    info_articles[eid] = {"ID_A": id_article,
                          "TITLE_A": infos.title,
                          "DOI": infos.doi,
                          "YEAR": infos.coverDate,
                          "AUTHORS": authors,
                          "ISSN": issn,
                          "ID_J": id_journal,
                          "TITLE_J": infos.publicationName,
                          "REFS": refs_eids,
                          "CITED_BY_COUNT": cited}
    
    gc.collect()
    
    os.remove('../../../Walter/.scopus/abstract_retrieval/' + eid)

with open('pickles/LOGIC_INFOS.pickle', 'wb') as f:
        pickle.dump([info_articles,info_journals], f)

Início da extração das informações do artigo 11243
Início da extração das informações do artigo 11244
Início da extração das informações do artigo 11245
Início da extração das informações do artigo 11246
Início da extração das informações do artigo 11247
Início da extração das informações do artigo 11248
Início da extração das informações do artigo 11249
Início da extração das informações do artigo 11250
Início da extração das informações do artigo 11251
Início da extração das informações do artigo 11252
Início da extração das informações do artigo 11253
Início da extração das informações do artigo 11254
Início da extração das informações do artigo 11255
Início da extração das informações do artigo 11256
Início da extração das informações do artigo 11257
Início da extração das informações do artigo 11258
Início da extração das informações do artigo 11259
Início da extração das informações do artigo 11260
Início da extração das informações do artigo 11261
Início da extração das informaç

Início da extração das informações do artigo 11404
Início da extração das informações do artigo 11405
Início da extração das informações do artigo 11406
Início da extração das informações do artigo 11407
Início da extração das informações do artigo 11408
Início da extração das informações do artigo 11409
Início da extração das informações do artigo 11410
Início da extração das informações do artigo 11411
Início da extração das informações do artigo 11412
Início da extração das informações do artigo 11413
Início da extração das informações do artigo 11414
Início da extração das informações do artigo 11415
Início da extração das informações do artigo 11416
Início da extração das informações do artigo 11417
Início da extração das informações do artigo 11418
Início da extração das informações do artigo 11419
Início da extração das informações do artigo 11420
Início da extração das informações do artigo 11421
Início da extração das informações do artigo 11422
Início da extração das informaç

Início da extração das informações do artigo 11565
Início da extração das informações do artigo 11566
Início da extração das informações do artigo 11567
Início da extração das informações do artigo 11568
Início da extração das informações do artigo 11569
Início da extração das informações do artigo 11570
Início da extração das informações do artigo 11571
Início da extração das informações do artigo 11572
Início da extração das informações do artigo 11573
Início da extração das informações do artigo 11574
Início da extração das informações do artigo 11575
Início da extração das informações do artigo 11576
Início da extração das informações do artigo 11577
Início da extração das informações do artigo 11578
Início da extração das informações do artigo 11579
Início da extração das informações do artigo 11580
Início da extração das informações do artigo 11581
Início da extração das informações do artigo 11582
Início da extração das informações do artigo 11583
Início da extração das informaç

Início da extração das informações do artigo 11726
Início da extração das informações do artigo 11727
Início da extração das informações do artigo 11728
Início da extração das informações do artigo 11729
Início da extração das informações do artigo 11730
Início da extração das informações do artigo 11731
Início da extração das informações do artigo 11732
Início da extração das informações do artigo 11733
Início da extração das informações do artigo 11734
Início da extração das informações do artigo 11735
Início da extração das informações do artigo 11736
Início da extração das informações do artigo 11737
Início da extração das informações do artigo 11738
Início da extração das informações do artigo 11739
Início da extração das informações do artigo 11740
Início da extração das informações do artigo 11741
Início da extração das informações do artigo 11742
Início da extração das informações do artigo 11743
Início da extração das informações do artigo 11744
Início da extração das informaç

Início da extração das informações do artigo 11887
Início da extração das informações do artigo 11888
Início da extração das informações do artigo 11889
Início da extração das informações do artigo 11890
Início da extração das informações do artigo 11891
Início da extração das informações do artigo 11892
Início da extração das informações do artigo 11893
Início da extração das informações do artigo 11894
Início da extração das informações do artigo 11895
Início da extração das informações do artigo 11896
Início da extração das informações do artigo 11897
Início da extração das informações do artigo 11898
Início da extração das informações do artigo 11899
Início da extração das informações do artigo 11900
Início da extração das informações do artigo 11901
Início da extração das informações do artigo 11902
Início da extração das informações do artigo 11903
Início da extração das informações do artigo 11904
Início da extração das informações do artigo 11905
Início da extração das informaç

...

Início da extração das informações do artigo 18005
Início da extração das informações do artigo 18006
Início da extração das informações do artigo 18007
Início da extração das informações do artigo 18008
Início da extração das informações do artigo 18009
Início da extração das informações do artigo 18010
Início da extração das informações do artigo 18011
Início da extração das informações do artigo 18012
Início da extração das informações do artigo 18013
Início da extração das informações do artigo 18014
Início da extração das informações do artigo 18015
Início da extração das informações do artigo 18016
Início da extração das informações do artigo 18017
Início da extração das informações do artigo 18018
Início da extração das informações do artigo 18019
Início da extração das informações do artigo 18020
Início da extração das informações do artigo 18021
Início da extração das informações do artigo 18022
Início da extração das informações do artigo 18023
Início da extração das informaç

Início da extração das informações do artigo 18166
Início da extração das informações do artigo 18167
Início da extração das informações do artigo 18168
Início da extração das informações do artigo 18169
Início da extração das informações do artigo 18170
Início da extração das informações do artigo 18171
Início da extração das informações do artigo 18172
Início da extração das informações do artigo 18173
Início da extração das informações do artigo 18174
Início da extração das informações do artigo 18175
Início da extração das informações do artigo 18176
Início da extração das informações do artigo 18177
Início da extração das informações do artigo 18178
Início da extração das informações do artigo 18179
Início da extração das informações do artigo 18180
Início da extração das informações do artigo 18181
Início da extração das informações do artigo 18182
Início da extração das informações do artigo 18183
Início da extração das informações do artigo 18184
Início da extração das informaç

Início da extração das informações do artigo 18327
Início da extração das informações do artigo 18328
Início da extração das informações do artigo 18329
Início da extração das informações do artigo 18330
Início da extração das informações do artigo 18331
Início da extração das informações do artigo 18332
Início da extração das informações do artigo 18333
Início da extração das informações do artigo 18334
Início da extração das informações do artigo 18335
Início da extração das informações do artigo 18336
Início da extração das informações do artigo 18337
Início da extração das informações do artigo 18338
Início da extração das informações do artigo 18339
Início da extração das informações do artigo 18340
Início da extração das informações do artigo 18341
Início da extração das informações do artigo 18342
Início da extração das informações do artigo 18343
Início da extração das informações do artigo 18344
Início da extração das informações do artigo 18345
Início da extração das informaç

Início da extração das informações do artigo 18488
Início da extração das informações do artigo 18489
Início da extração das informações do artigo 18490
Início da extração das informações do artigo 18491
Início da extração das informações do artigo 18492
Início da extração das informações do artigo 18493
Início da extração das informações do artigo 18494
Início da extração das informações do artigo 18495
Início da extração das informações do artigo 18496
Início da extração das informações do artigo 18497
Início da extração das informações do artigo 18498
Início da extração das informações do artigo 18499
Início da extração das informações do artigo 18500
Início da extração das informações do artigo 18501
Início da extração das informações do artigo 18502
Início da extração das informações do artigo 18503
Início da extração das informações do artigo 18504
Início da extração das informações do artigo 18505
Início da extração das informações do artigo 18506
Início da extração das informaç

Início da extração das informações do artigo 18649
Início da extração das informações do artigo 18650
Início da extração das informações do artigo 18651
Início da extração das informações do artigo 18652
Início da extração das informações do artigo 18653
Início da extração das informações do artigo 18654
Início da extração das informações do artigo 18655
Início da extração das informações do artigo 18656
Início da extração das informações do artigo 18657
Início da extração das informações do artigo 18658
Início da extração das informações do artigo 18659
Início da extração das informações do artigo 18660
Início da extração das informações do artigo 18661
Início da extração das informações do artigo 18662
Início da extração das informações do artigo 18663
Início da extração das informações do artigo 18664
Início da extração das informações do artigo 18665
Início da extração das informações do artigo 18666
Início da extração das informações do artigo 18667
Início da extração das informaç

In [26]:
# with open('pickles/LOGIC_INFOS.pickle', 'wb') as f:
#     pickle.dump([info_articles,info_journals], f)

## 4. Usando a API do Scopus, extrair e armazenar as informações das referências de cada artigo.

In [68]:
with open('pickles/LOGIC_INFOS.pickle', 'rb') as f:
    [info_articles, info_journals] = pickle.load(f)

In [58]:
print('Total de Periódicos: ' + str(len(info_journals)))

Total de Periódicos: 18


In [59]:
print('Total de Artigos publicados na área de Lógica: ' + str(len(info_articles)))

Total de Artigos publicados na área de Lógica: 18718


In [63]:
keys = createKeys(password,email)

C:\Users\Walter\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)


In [64]:
changeKeys(0,keys)

Nova Chave: 89b6784f6cf030d189da29ac5cd60e78


In [114]:
# info_journals = {}
# info_articles = {}
#ref_error = []

with open('pickles/LOGIC_REFS.pickle', 'rb') as f:
    [info_articles, info_journals,ref_error] = pickle.load(f)
    
#limit = 0
#k = 1
#art = 0
ini = all_source_eids.index(eid)
art = art - 1

for eid in all_source_eids[ini:]:
    
    REFS = info_articles[eid]['REFS']
    
    art = art +  1
    print('Início da extração das informações das citações do artigo ' + str(art))
    cit = 0
    
    if len(REFS) != 0:
    
        for ref in REFS:
            
            cit = cit + 1

            if k == 10:
                with open('pickles/LOGIC_REFS.pickle', 'wb') as f:
                    pickle.dump([info_articles,info_journals,ref_error], f)
                print('>>> Geração de Novas Chaves!')
                keys = createKeys(password,email)
                k = 0
                limit = 0
                print('>>> Troca de Chave: ' + str(k))
                changeKeys(k,keys)

            elif limit >= (10000-20):
                with open('pickles/LOGIC_REFS.pickle', 'wb') as f:
                    pickle.dump([info_articles,info_journals,ref_error], f)
                k = k + 1
                limit = 0
                if k == 10: continue
                print('>>> Troca de Chave: ' + str(k))
                changeKeys(k,keys)
                
            
            if info_articles.get(ref) == None:

                id_article = len(info_articles) + 1

                try:
                    infos = sc.AbstractRetrieval(ref , view='FULL', refresh=True)
                except sc.exception.Scopus404Error:
                    try:
                        time.sleep(14)
                        infos = sc.AbstractRetrieval(ref , view='FULL', refresh=True)
                    except sc.exception.Scopus404Error:
                        ref_error.append(ref)
                        continue

                limit = limit + 1

                if infos.issn != None:

                    aux_issn = infos.issn.split(' ')
                    m=0
                    n=1
                    issn = aux_issn[0]

                    if len(aux_issn) == 1: 
                        n = 0

                    if info_journals.get(aux_issn[m]) == None and info_journals.get(aux_issn[n]) == None:

                        id_journal = len(info_journals) + 1
                        info_journals[issn] = {"ID_J": id_journal,
                                               "TITLE_J": infos.publicationName}

                    elif info_journals.get(aux_issn[0]) != None:
                        id_journal = info_journals[issn]['ID_J']

                    else:
                        issn = aux_issn[1]
                        id_journal = info_journals[issn]['ID_J']

                else: 
                    issn = infos.publicationName
                    if info_journals.get(issn) == None:
                        id_journal = len(info_journals) + 1
                        info_journals[issn] = {"ID_J": id_journal,
                                               "TITLE_J": infos.publicationName}
                    else:
                        id_journal = info_journals[issn]['ID_J']


                refs = infos.references
                refs_eids = []
                if refs != None:
                    for r in range(len(refs)):
                        refs_eids.append(refs[r][1])

                authors = {}
                if infos.authors != None:
                    for a in range(len(infos.authors)):
                        authors['author_' + str(a)] ={'auid': infos.authors[a][0],
                                                      'indexed_name': infos.authors[a][1],
                                                      'affiliation': infos.authors[a][-1]}

                try: 
                    cited = infos.citedby_count
                except KeyError:
                    cited = 'NaN'

                info_articles[ref] = {"ID_A": id_article,
                                      "TITLE_A": infos.title,
                                      "DOI": infos.doi,
                                      "YEAR": infos.coverDate,
                                      "AUTHORS": authors,
                                      "ISSN": issn,
                                      "ID_J": id_journal,
                                      "TITLE_J": infos.publicationName,
                                      "REFS": refs_eids,
                                      "CITED_BY_COUNT": cited}

                gc.collect()
                os.remove('../../../Walter/.scopus/abstract_retrieval/' + ref)
                print('>>> Citação ' + str(cit))
            
        with open('pickles/LOGIC_REFS.pickle', 'wb') as f:
            pickle.dump([info_articles,info_journals,ref_error], f)

Início da extração das informações das citações do artigo 15671
>>> Citação 43
>>> Citação 44
>>> Citação 45
>>> Citação 46
>>> Citação 47
>>> Citação 50
>>> Citação 51
>>> Citação 52
>>> Citação 53
>>> Citação 54
>>> Citação 55
>>> Citação 56
>>> Citação 57
>>> Citação 59
>>> Citação 60
>>> Citação 62
>>> Citação 63
>>> Citação 64
>>> Citação 65
>>> Citação 66
>>> Citação 67
>>> Citação 69
>>> Citação 70
>>> Citação 71
>>> Citação 73
>>> Citação 75
>>> Citação 76
>>> Citação 77
>>> Citação 79
>>> Citação 81
>>> Citação 82
>>> Citação 83
>>> Citação 84
>>> Citação 86
>>> Citação 87
>>> Citação 89
>>> Citação 91
>>> Citação 95
>>> Citação 101
>>> Citação 103
>>> Citação 105
>>> Citação 107
>>> Citação 109
>>> Citação 110
>>> Citação 111
>>> Citação 112
Início da extração das informações das citações do artigo 15672
>>> Citação 1
>>> Citação 9
>>> Citação 11
Início da extração das informações das citações do artigo 15673
>>> Citação 2
>>> Citação 9
>>> Citação 11
>>> Citação 12
>>> Citaç

>>> Citação 25
>>> Citação 26
Início da extração das informações das citações do artigo 15720
>>> Citação 1
>>> Citação 2
>>> Citação 3
>>> Citação 4
>>> Citação 5
>>> Citação 7
>>> Citação 9
>>> Citação 12
>>> Citação 13
>>> Citação 14
>>> Citação 16
>>> Citação 17
>>> Citação 18
>>> Citação 19
>>> Citação 21
>>> Citação 22
>>> Citação 23
>>> Citação 24
>>> Citação 25
>>> Citação 26
>>> Citação 27
>>> Citação 28
Início da extração das informações das citações do artigo 15721
>>> Citação 1
Início da extração das informações das citações do artigo 15722
>>> Citação 1
>>> Citação 2
>>> Citação 11
>>> Citação 12
Início da extração das informações das citações do artigo 15723
>>> Citação 5
>>> Citação 10
Início da extração das informações das citações do artigo 15724
>>> Citação 7
>>> Citação 9
>>> Citação 12
Início da extração das informações das citações do artigo 15725
>>> Citação 4
>>> Citação 12
Início da extração das informações das citações do artigo 15726
>>> Citação 1
>>> Citação 

>>> Citação 1
>>> Citação 2
>>> Citação 3
>>> Citação 6
>>> Citação 8
>>> Citação 9
>>> Citação 10
>>> Citação 11
>>> Citação 13
>>> Citação 15
Início da extração das informações das citações do artigo 15757
>>> Citação 4
>>> Citação 13
>>> Citação 16
Início da extração das informações das citações do artigo 15758
>>> Citação 5
>>> Citação 6
>>> Citação 7
>>> Citação 8
>>> Citação 12
>>> Citação 13
Início da extração das informações das citações do artigo 15759
>>> Citação 2
Início da extração das informações das citações do artigo 15760
>>> Citação 8
>>> Citação 9
>>> Citação 15
>>> Citação 17
>>> Citação 19
>>> Citação 20
>>> Citação 24
>>> Citação 30
Início da extração das informações das citações do artigo 15761
Início da extração das informações das citações do artigo 15762
>>> Citação 1
>>> Citação 3
>>> Citação 4
>>> Citação 5
>>> Citação 6
>>> Citação 7
>>> Citação 8
>>> Citação 9
>>> Citação 10
>>> Citação 12
>>> Citação 14
>>> Citação 15
>>> Citação 16
>>> Citação 17
>>> Cita

>>> Citação 3
>>> Citação 4
Início da extração das informações das citações do artigo 15808
>>> Citação 5
>>> Citação 7
>>> Citação 11
>>> Citação 16
>>> Citação 18
>>> Citação 21
>>> Citação 23
>>> Citação 24
>>> Citação 25
>>> Citação 26
Início da extração das informações das citações do artigo 15809
>>> Citação 4
>>> Citação 5
>>> Citação 7
>>> Citação 14
>>> Citação 18
>>> Citação 21
Início da extração das informações das citações do artigo 15810
>>> Citação 1
>>> Citação 2
>>> Citação 3
>>> Citação 4
>>> Citação 5
>>> Citação 6
>>> Citação 7
>>> Citação 9
>>> Citação 10
>>> Citação 11
>>> Citação 12
>>> Citação 13
>>> Citação 14
>>> Citação 15
>>> Citação 16
>>> Citação 18
>>> Citação 19
>>> Citação 20
>>> Citação 21
>>> Citação 22
>>> Citação 23
>>> Citação 24
>>> Citação 25
>>> Citação 26
>>> Citação 27
>>> Citação 28
>>> Citação 29
>>> Citação 30
>>> Citação 31
>>> Citação 32
>>> Citação 33
>>> Citação 34
>>> Citação 35
>>> Citação 36
>>> Citação 37
>>> Citação 38
>>> Citação 3

>>> Citação 2
>>> Citação 4
Início da extração das informações das citações do artigo 15849
>>> Citação 3
>>> Citação 4
>>> Citação 8
Início da extração das informações das citações do artigo 15850
Início da extração das informações das citações do artigo 15851
>>> Citação 5
Início da extração das informações das citações do artigo 15852
Início da extração das informações das citações do artigo 15853
Início da extração das informações das citações do artigo 15854
>>> Citação 2
>>> Citação 3
>>> Citação 4
>>> Citação 9
>>> Citação 12
>>> Citação 15
Início da extração das informações das citações do artigo 15855
>>> Citação 4
Início da extração das informações das citações do artigo 15856
>>> Citação 8
>>> Citação 9
>>> Citação 10
>>> Citação 11
>>> Citação 13
>>> Citação 15
>>> Citação 18
>>> Citação 25
>>> Citação 28
>>> Citação 31
>>> Citação 32
>>> Citação 40
>>> Citação 41
>>> Citação 44
>>> Citação 45
>>> Citação 47
>>> Citação 48
>>> Citação 49
>>> Citação 50
>>> Citação 52
>>> Ci

...

>>> Citação 194
>>> Citação 195
>>> Citação 196
>>> Citação 197
>>> Citação 199
>>> Citação 200
>>> Citação 201
>>> Citação 202
>>> Citação 203
>>> Citação 204
>>> Citação 205
>>> Citação 206
>>> Citação 207
>>> Citação 208
>>> Citação 210
>>> Citação 211
>>> Citação 212
>>> Citação 213
>>> Citação 214
>>> Citação 216
>>> Citação 217
>>> Citação 218
>>> Citação 219
>>> Citação 220
>>> Citação 221
>>> Citação 222
>>> Citação 224
>>> Citação 225
>>> Citação 226
>>> Citação 227
>>> Citação 228
>>> Citação 229
>>> Citação 230
>>> Citação 231
>>> Citação 232
>>> Citação 234
>>> Citação 235
>>> Citação 236
>>> Citação 240
>>> Citação 241
>>> Citação 242
>>> Citação 243
>>> Citação 244
>>> Citação 245
>>> Citação 246
>>> Citação 247
>>> Citação 248
>>> Citação 249
>>> Citação 250
>>> Citação 252
>>> Citação 253
>>> Citação 254
>>> Citação 256
>>> Citação 257
>>> Citação 258
>>> Citação 259
>>> Citação 260
>>> Citação 262
>>> Citação 263
>>> Citação 264
>>> Citação 265
>>> Citação 267
Início d

>>> Citação 6
Início da extração das informações das citações do artigo 18535
>>> Citação 3
>>> Citação 4
>>> Citação 7
>>> Citação 8
>>> Citação 10
>>> Citação 12
Início da extração das informações das citações do artigo 18536
>>> Citação 4
>>> Citação 5
>>> Citação 6
Início da extração das informações das citações do artigo 18537
>>> Citação 4
Início da extração das informações das citações do artigo 18538
>>> Citação 9
>>> Citação 10
Início da extração das informações das citações do artigo 18539
Início da extração das informações das citações do artigo 18540
>>> Citação 3
>>> Citação 5
>>> Citação 6
>>> Citação 11
>>> Citação 14
>>> Citação 16
>>> Citação 18
>>> Citação 19
Início da extração das informações das citações do artigo 18541
>>> Citação 4
Início da extração das informações das citações do artigo 18542
>>> Citação 6
>>> Citação 16
>>> Citação 17
>>> Citação 18
>>> Citação 24
Início da extração das informações das citações do artigo 18543
Início da extração das informações

>>> Citação 1
>>> Citação 2
>>> Citação 4
Início da extração das informações das citações do artigo 18597
>>> Citação 4
>>> Citação 12
>>> Citação 13
Início da extração das informações das citações do artigo 18598
>>> Citação 3
>>> Citação 5
>>> Citação 6
>>> Citação 7
>>> Citação 8
>>> Citação 9
>>> Citação 10
>>> Citação 11
>>> Citação 14
>>> Citação 15
>>> Citação 17
>>> Citação 18
>>> Citação 19
>>> Citação 21
>>> Citação 23
>>> Citação 24
>>> Citação 25
Início da extração das informações das citações do artigo 18599
>>> Citação 1
>>> Citação 2
>>> Citação 7
>>> Citação 8
>>> Citação 13
>>> Citação 14
>>> Citação 15
>>> Citação 17
>>> Citação 19
Início da extração das informações das citações do artigo 18600
>>> Citação 5
Início da extração das informações das citações do artigo 18601
>>> Citação 9
Início da extração das informações das citações do artigo 18602
>>> Citação 2
>>> Citação 5
>>> Citação 7
>>> Citação 9
Início da extração das informações das citações do artigo 18603
In

>>> Citação 2
>>> Citação 3
>>> Citação 5
>>> Citação 7
>>> Citação 11
>>> Citação 12
>>> Citação 13
>>> Citação 14
Início da extração das informações das citações do artigo 18645
>>> Citação 6
>>> Citação 11
Início da extração das informações das citações do artigo 18646
>>> Citação 1
>>> Citação 5
>>> Citação 6
>>> Citação 10
>>> Citação 12
Início da extração das informações das citações do artigo 18647
>>> Citação 2
>>> Citação 5
>>> Citação 8
Início da extração das informações das citações do artigo 18648
>>> Citação 1
>>> Citação 3
>>> Citação 11
Início da extração das informações das citações do artigo 18649
>>> Citação 5
>>> Citação 7
Início da extração das informações das citações do artigo 18650
Início da extração das informações das citações do artigo 18651
>>> Citação 1
>>> Citação 2
>>> Citação 3
>>> Citação 4
>>> Citação 5
>>> Citação 6
>>> Citação 7
>>> Citação 8
>>> Citação 9
>>> Citação 10
>>> Citação 11
>>> Citação 12
>>> Citação 13
>>> Citação 14
>>> Citação 15
>>> Ci

>>> Citação 6
>>> Citação 10
>>> Citação 11
>>> Citação 22
>>> Citação 23
>>> Citação 24
>>> Citação 25
>>> Citação 30
>>> Citação 31
>>> Citação 33
Início da extração das informações das citações do artigo 18684
>>> Citação 3
>>> Citação 8
>>> Citação 22
>>> Citação 23
>>> Citação 30
>>> Citação 32
Início da extração das informações das citações do artigo 18685
>>> Citação 1
>>> Citação 3
>>> Citação 4
>>> Citação 7
>>> Citação 8
>>> Citação 10
>>> Citação 14
>>> Citação 17
>>> Citação 20
>>> Citação 21
>>> Citação 23
>>> Citação 24
>>> Citação 27
>>> Citação 29
>>> Citação 31
Início da extração das informações das citações do artigo 18686
>>> Citação 2
>>> Citação 10
>>> Citação 11
>>> Citação 13
>>> Citação 16
>>> Citação 18
>>> Citação 19
>>> Citação 20
>>> Citação 21
>>> Citação 22
Início da extração das informações das citações do artigo 18687
Início da extração das informações das citações do artigo 18688
>>> Citação 2
>>> Citação 3
>>> Citação 7
>>> Citação 15
>>> Citação 16
>>

In [115]:
with open('pickles/LOGIC_REFS.pickle', 'wb') as f:
    pickle.dump([info_articles,info_journals,ref_error], f)

In [117]:
len(ref_error)

27

In [118]:
len(info_articles)

187938

In [120]:
len(info_journals)

52267

In [121]:
infos = sc.AbstractRetrieval(ref , view='FULL', refresh=True)

In [ ]:
infos.